In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
details = pd.read_csv("Data/games_details.csv", low_memory=False)

In [ ]:
bad_games = (
    details
    .groupby("GAME_ID")["START_POSITION"]
    .apply(lambda x: x.notna().sum())
    .loc[lambda x: x != 10]
)
bad_game_ids = bad_games.index
details[details["GAME_ID"].isin(bad_game_ids)]

In [ ]:
# START_POSITION 확인
bad_games = (
    details
    .groupby("GAME_ID")["START_POSITION"]
    .apply(lambda x: x.notna().sum())
    .loc[lambda x: x != 10]
)
bad_games

In [ ]:
bad_games.value_counts()

In [ ]:
details.info()

In [ ]:
details.describe()

In [ ]:
details.head()

In [ ]:
# GSW의 자료에 null값 확인 -> 전부 부상이나 특정 이유로 불참.
# Null값은 제외하고 생각. 참여자체를 안했기에.
cols_to_check = ["MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT",
                 "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB",
                 "AST", "STL", "BLK", "TO", "PF", "PTS"]
null_condition = details[cols_to_check].isna().any(axis=1)
gsw_nulls = details[null_condition & (details["TEAM_ABBREVIATION"] == "GSW")]
(
    gsw_nulls["COMMENT"]
    .notna()
    & gsw_nulls["COMMENT"].str.strip().ne("")
).all()

## Details 테이블 전처리
### 전처리 필요가 없는 칼럼
- GAME_ID, TEAM_ID,TEAM_ABBREVIATION, PLAYER_ID, PLAYER_NAME

### 전처리 필요한 칼럼
- START_POSITION(카테고리 화, null처리), MIN(min.decimal, null 처리), PLUS_MINUS(Null값 처리), FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT,OREB, DREB, REB, AST, STL, BLK, TO, PF, PTS(Null 처리)

In [ ]:
# clean_details 생성
clean_details = details.copy()

In [ ]:
# 필요한 칼럼들만 선택
clean_details = clean_details[
    ["GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION", "PLAYER_ID", "PLAYER_NAME",
     "START_POSITION", "MIN", "PLUS_MINUS", "FGM", "FGA", "FG_PCT", "FG3M",
     "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB", "AST",
     "STL", "BLK", "TO", "PF", "PTS"]
].copy()

In [ ]:
# Start_position 카테고리화
clean_details["START_POSITION"] = clean_details["START_POSITION"].astype("category")

In [ ]:
# Start_position 처리되었는지 확인
clean_details["START_POSITION"].dtype

In [ ]:
# MIN 처리 전 확인 1
clean_details["MIN"].apply(type).value_counts()

In [ ]:
# MIN 처리 전 확인 2
clean_details[
    clean_details["MIN"].apply(type) == str]["MIN"].value_counts().head(10)

In [ ]:
# MIN 처리 전 확인 3
clean_details["MIN"].value_counts(dropna=False)

In [ ]:
# MIN into float
def minutes_to_decimal(x):
    if pd.isna(x):
        return pd.NA

    if isinstance(x, (int, float)):
        return float(x)

    s = str(x).strip()

    if ":" in s:
        mins, secs = s.split(":")
        return float(mins) + float(secs) / 60

    return float(s)

clean_details["MIN_DECIMAL"] = clean_details["MIN"].apply(minutes_to_decimal)


In [ ]:
# 경기에 참여하지 않은 선수들의 데이터에 대한 자료는 drop
cols_to_check = ["MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT",
                 "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB",
                 "AST", "STL", "BLK", "TO", "PF", "PTS", "PLUS_MINUS"]
clean_details = clean_details.dropna(subset=cols_to_check, how="all")

In [ ]:
# drop한 결과 확인
before = len(details)
after = len(clean_details)
print(f"{before - after} rows dropped.")

In [ ]:
# PLUS_MINUS 칼럼 확인
clean_details[clean_details["PLUS_MINUS"].isnull()]

In [ ]:
# PLUS_MINUS 칼럼 중에 가능한 것들만 나타내는 파생칼럼 생성.
clean_details["PLUS_MINUS_AVAILABLE"] = clean_details["PLUS_MINUS"].notna()
clean_details["PLUS_MINUS_AVAILABLE"].value_counts()

In [ ]:
# PLUS_MINUS 칼럼 평균이 0에 근접한 것을 확인함.
clean_details["PLUS_MINUS"].describe()

In [ ]:
clean_details["PLUS_MINUS"] = clean_details["PLUS_MINUS"].fillna(0)
clean_details["PLUS_MINUS"].isnull().sum()

## games 테이블 전처리
### 전처리 필요가 없는 칼럼
GAME_ID, HOME_TEAM_ID, VISITOR_TEAM_ID
### 전처리 필요한 칼럼
- GAME_DATE_EST(date), PTS_home(Null처리), PTS_away(Null처리), SEASON(Null처리)

In [ ]:
# clean_games 생성
clean_games= games.copy()

In [ ]:
# 필요한 칼럼들만 선택
clean_games = clean_games[
    ["GAME_ID", "GAME_DATE_EST", "HOME_TEAM_ID", "VISITOR_TEAM_ID",
     "PTS_home", "PTS_away", "SEASON"]
]

In [ ]:
clean_games.info()

In [ ]:
# PTS_home 확인 1
clean_games[clean_games["PTS_home"].isnull()]

In [ ]:
# PTS_home 확인 2
bad_game_ids = games[
    clean_games.filter(like="_home").isna().all(axis=1)
]["GAME_ID"]

clean_details[clean_details["GAME_ID"].isin(bad_game_ids)].shape

In [ ]:
# PTS_home, PTS_away, SEASON null인 것들을 drop
# 경기자체의 데이터가 없는 빈껍질 스케줄이기 때문에 drop
clean_games = clean_games.dropna()

In [ ]:
# drop 결과 확인
before = len(games)
after = len(clean_games)
print(f"{before - after} rows dropped.")

### 주제: “우승 시즌에는 상대팀이 달라져도 선수들이 비슷한 수준의 활약을 했을까?”(Golden State Warriors vs.)
#### 활약은 어떻게 정의할 것인가?
- 활약: 선수의 득점(PTS, 보조지표로 FG_PCT / FG3_PCT / FT_PCT사용)
- 활약: PLUS_MINUS, 전반적인 영향력.
- PLUS-MINUS는 선수가 코트에 있을 때 팀의 득실점 차이를 직접적으로 반영하기 때문에, 박스스코어 기반 지표 중에서는 선수의 전반적인 경기 영향력을 가장 직관적으로 관찰할 수 있는 지표입니다. 특히나, 득점, 수비, 볼 흐름 등 개별 스탯으로 분리하기 어려운 요소들이 모두 반영된 상태에서 팀이 얼마나 더 잘했는지를 보여줍니다.
- 선수: MIN을 기준으로 많은 선수 선택(경기에 적은 시간을 뛴 선수는 분석결과를 흐릴 수 있음)
- 비교년도와 비교대상년도들에 둘 다 존재하는 선수의 경우 비교가 가능하다.
- 본 분석에서는 ‘활약’을 단일 정의로 한정하지 않고, 득점, 전반적 영향력의 두 가지 정의에서 실시한다.

### 활약을 선수의 득점으로 정의하고 분석시행

In [ ]:
# GSW_details 테이블 생성
GSW_details = clean_details[clean_details["TEAM_ABBREVIATION"]=="GSW"]
GSW_details

In [ ]:
# 시즌별 게임 아이디 테이블 생성
SEASON_games =clean_games[["GAME_ID","SEASON",'GAME_DATE_EST']]

In [ ]:
# GSW_details 테이블에 merge
GSW_details_merged = GSW_details.merge(SEASON_games, on="GAME_ID", how="inner")
GSW_details_merged

In [ ]:
# Deciding the Championship Year and comparing years.
champ_season = 2014
pre_seasons = [2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006,2005,2004,2003]
GSW_Champyear= GSW_details_merged[GSW_details_merged["SEASON"]==champ_season]
GSW_Otheryear= GSW_details_merged[GSW_details_merged["SEASON"].isin(pre_seasons)]

In [ ]:
# Getting the Team ID of Golden State Warriors.
if "TEAM_ABBREVIATION" in GSW_Champyear.columns:
    gsw_ids = GSW_Champyear.loc[GSW_Champyear["TEAM_ABBREVIATION"]=="GSW", "TEAM_ID"].dropna().unique()
else:
    gsw_ids = GSW_Champyear["TEAM_ID"].dropna().unique()
GSW_TEAM_ID = int(gsw_ids[0])

In [ ]:
# Creating an opponent column for both years.
games_map = games[["GAME_ID","HOME_TEAM_ID","VISITOR_TEAM_ID"]].copy()

def add_opponent(df):
    out = df.merge(games_map, on="GAME_ID", how="left")
    out["OPP_TEAM_ID"] = np.where(
        out["HOME_TEAM_ID"] == GSW_TEAM_ID,
        out["VISITOR_TEAM_ID"],
        out["HOME_TEAM_ID"]
    )
    out["GSW_HOME_AWAY"] = np.where(out["HOME_TEAM_ID"] == GSW_TEAM_ID, "HOME", "AWAY")
    return out
GSW_Champyear = add_opponent(GSW_Champyear)
GSW_Otheryear = add_opponent(GSW_Otheryear)

In [ ]:
# Defining the number of top players to consider.
TOP_N = 5
core_players = (
    GSW_Champyear.groupby(["PLAYER_ID","PLAYER_NAME"], as_index=False)["MIN_DECIMAL"].sum()
    .sort_values("MIN_DECIMAL", ascending=False)
    .head(TOP_N)
)
core_ids = core_players["PLAYER_ID"].unique()
Champ_core = GSW_Champyear[GSW_Champyear["PLAYER_ID"].isin(core_ids)].copy()
Other_core = GSW_Otheryear[GSW_Otheryear["PLAYER_ID"].isin(core_ids)].copy()

In [ ]:
# Defining a function to summarize player performance against opponents.
def player_vs_opp_summary(df):
    return (
        df.groupby(["PLAYER_ID","PLAYER_NAME","OPP_TEAM_ID"], as_index=False)
          .agg(
              games_vs_opp=("GAME_ID","nunique"),
              pts_mean=("PTS","mean"),
              fg_mean=("FG_PCT","mean"),
              fg3_mean=("FG3_PCT","mean"),
              ft_mean=("FT_PCT","mean"),
          )
    )
Champ_player_opp = player_vs_opp_summary(Champ_core)
Other_player_opp = player_vs_opp_summary(Other_core)

In [ ]:
# Players with at least 2 games against them.
MIN_GAMES_VS_OPP = 3
Champ_player_opp_over2 = Champ_player_opp[Champ_player_opp["games_vs_opp"] >= MIN_GAMES_VS_OPP].copy()
Other_player_opp_over2 = Other_player_opp[Other_player_opp["games_vs_opp"] >= MIN_GAMES_VS_OPP].copy()

In [ ]:
Champ_player_opp_over2.head()

In [ ]:
Other_player_opp_over2.head()

In [ ]:
# Compute “stability across opponents” per player
def opponent_stability(df_player_opp):
    return (
        df_player_opp.groupby(["PLAYER_ID","PLAYER_NAME"], as_index=False)
            .agg(
                num_opponents_faced=("OPP_TEAM_ID","nunique"),
                pts_전체_평균=("pts_mean","mean"),
                pts_표준편차=("pts_mean","std"),
                pts_최소값=("pts_mean","min"),
                pts_최대값=("pts_mean","max"),
                fg_표준편차=("fg_mean","std"),
                fg3_표준편차=("fg3_mean","std"),
                ft_표준편차=("ft_mean","std"),
            )
    )
Champ_stability = opponent_stability(Champ_player_opp_over2)
Other_stability = opponent_stability(Other_player_opp_over2)

Champ_stability.head()

In [ ]:
# Plot style change
plt.rcdefaults()
plt.style.use("seaborn-v0_8-white")
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["axes.prop_cycle"] = cycler(color=["#1D428A", "#FFC72C", "#26282A", "#E03A3E"])

In [ ]:
Champ_plot = Champ_stability[["PLAYER_NAME", "pts_전체_평균", "pts_표준편차"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Champ_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("우승 시즌 선수 득점 지표")
plt.tight_layout()
plt.show()

- 상대별 평균 득점의 평균(pts_mean_overall)과 상대별 평균 득점의 표준편차(pts_std)를 비교해봤을 때 상대적으로 pts_std가 큰 것을 볼 수 있음.
- 이는 선수의 활약이 상대팀이 달라짐에 따라 안정적이지 못한다고 해석해 볼 수 있음.

In [ ]:
Champ_plot = Champ_stability[["PLAYER_NAME", "pts_최소값","pts_최대값"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Champ_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("우승 시즌 상대별 최소, 최대득점")
plt.tight_layout()
plt.show()

- 상대별 평균득점의 최소점과 최대점이 차이가 생각보다 나는 것처럼 보인다.
- 이는 선수의 활약이 상대에 의존이 큰 매치업이 존재한다는 것을 볼 수 있다.

In [ ]:
Champ_plot = Champ_stability[
    ["PLAYER_NAME", "fg_표준편차", "fg3_표준편차", "ft_표준편차"]
].set_index("PLAYER_NAME")
fig, ax = plt.subplots(figsize=(12, 6))
Champ_plot.T.plot(
    kind="bar",
    ax=ax,
    width=0.8
)
ax.set_ylabel("표준편차값")
ax.set_title("우승 시즌 선수들의 슛 유형별 득점 변화")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

ax.legend(
    title="Player",
    bbox_to_anchor=(1.02, 1),
    loc="upper left"
)
plt.tight_layout()
plt.show()

- 대체적으로 상대팀별 field goal 평균과 3점슛 평균, 자유투 평균의 분산은 작은 것으로 확인할 수 있다.
- 이는 선수들이 상대방이 변화하더라도, 안정적인 field goal, 3점슛, 자유투를 했다고 해석해볼 수 있다.

In [ ]:
Other_stability.head()

In [ ]:
Other_plot = Other_stability[["PLAYER_NAME","pts_전체_평균", "pts_표준편차"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Other_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("이전 시즌 선수 득점 지표")
plt.tight_layout()
plt.show()

- 상대별 평균 득점의 평균(pts_mean_overall)과 상대별 평균 득점의 표준편차(pts_std)를 비교해봤을 때 상대적으로 pts_std가 작은 것을 볼 수 있음.
- 이는 선수의 활약이 상대팀이 달라짐에도 비교적 안정적이라고 해석해볼 수 있다.

In [ ]:
Other_plot = Other_stability[["PLAYER_NAME", "pts_최소값","pts_최대값"]].set_index("PLAYER_NAME")
plt.figure(figsize=(12,12))
Other_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("이전 시즌 상대별 최소, 최대득점")
plt.tight_layout()
plt.show()

- 상대별 평균득점의 최소점과 최대점이 차이가 생각보다 크게 나는 것처럼 보인다.
- 이는 선수의 활약이 상대에 의존이 큰 매치업이 존재한다는 것을 볼 수 있다.

In [ ]:
Other_plot = Other_stability[
    ["PLAYER_NAME", "fg_표준편차", "fg3_표준편차", "ft_표준편차"]
].set_index("PLAYER_NAME")
fig, ax = plt.subplots(figsize=(12, 6))
Other_plot.T.plot(
    kind="bar",
    ax=ax,
    width=0.8
)
ax.set_ylabel("표준편차값")
ax.set_title("이전 시즌 선수들의 슛 유형별 득점 변화")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

ax.legend(
    title="Player",
    bbox_to_anchor=(1.02, 1),
    loc="upper left"
)

plt.tight_layout()
plt.show()

- 대체적으로 상대팀별 field goal 평균과 3점슛 평균, 자유투 평균의 분산은 작은 것으로 확인할 수 있다.
- 이는 선수들이 상대방이 변화하더라도, 안정적인 field goal, 3점슛, 자유투를 했다고 해석해볼 수 있다.

In [ ]:
# Now compare the results of other years and champ year
compare = (
    Champ_stability.merge(
        Other_stability,
        on=["PLAYER_ID","PLAYER_NAME"],
        how="inner",
        suffixes=("_champ","_other")
    )
)
compare["delta_pts_표준편차"] = compare["pts_표준편차_champ"] - compare["pts_표준편차_other"]
compare["delta_fg_표준편차"] = compare["fg_표준편차_champ"] - compare["fg_표준편차_other"]
compare["delta_fg3_표준편차"] = compare["fg3_표준편차_champ"] - compare["fg3_표준편차_other"]
compare["delta_ft_표준편차"] = compare["ft_표준편차_champ"] - compare["ft_표준편차_other"]
compare

In [ ]:
compare_plot = compare[["PLAYER_NAME", "delta_pts_표준편차", "delta_fg_표준편차", "delta_fg3_표준편차", "delta_ft_표준편차"]].set_index("PLAYER_NAME")
plt.figure(figsize=(12,12))
compare_plot.T.plot(kind="bar", rot=0)
plt.ylabel("표준편차 차이")
plt.title("우승시즌 - 이전시즌 슛 유형별 선수 득점 안정성")
plt.tight_layout()
plt.show()

- 2015년 시즌과 이전 시즌을 비교했을 때 대부분 선수가 상대별 득점 평균의 분산이 증가했다고 알 수 있다.
- 이는 선수의 활약이 이전시즌과 비교해, 우승시즌에는 상대팀이 달라지면 안정적이지 못하다는 것을 볼 수 있다.
- 또한 field goal 평균과 3점슛 평균, 자유투 평균의 분산에 변화가 적다는 것을 볼 수 있다.
- 안정적으로 field goal, 3점슛, 자유투를 유지하였다는 것으로, 위에서 볼 수 있었던 득점 안정성의 부재는 선수의 기량의 문제가 아니라는 것을 확인해볼 수 있다.

### 활약을 선수의 PLUS_MINUS로 정의하고 분석시행

In [ ]:
# Defining the number of top players to consider.
# TOP_N = 5
# core_players = (
#     GSW_Champyear.groupby(["PLAYER_ID","PLAYER_NAME"], as_index=False)["MIN"].sum()
#     .sort_values("MIN", ascending=False)
#     .head(TOP_N)
# )
# core_ids = core_players["PLAYER_ID"].unique()
# Champ_core = GSW_Champyear[GSW_Champyear["PLAYER_ID"].isin(core_ids)].copy()
# Other_core = GSW_Otheryear[GSW_Otheryear["PLAYER_ID"].isin(core_ids)].copy()

In [ ]:
# Getting the player vs opponent summary with plus-minus
def player_vs_opp_summary_pm(df):
    return (
        df.groupby(["PLAYER_ID","PLAYER_NAME","OPP_TEAM_ID"], as_index=False)
          .agg(
              games_vs_opp=("GAME_ID","nunique"),
              pm_mean=("PLUS_MINUS","mean"),))
Champ_player_opp_pm = player_vs_opp_summary_pm(Champ_core)
Other_player_opp_pm = player_vs_opp_summary_pm(Other_core)

In [ ]:
# Limiting to players with at least 2 games against them.
MIN_GAMES_VS_OPP = 2
Champ_player_opp_pm_over2 = Champ_player_opp_pm[Champ_player_opp_pm["games_vs_opp"] >= MIN_GAMES_VS_OPP].copy()
Other_player_opp_pm_over2 = Other_player_opp_pm[Other_player_opp_pm["games_vs_opp"] >= MIN_GAMES_VS_OPP].copy()

In [ ]:
Champ_player_opp_pm_over2.head()

In [ ]:
Other_player_opp_pm_over2.head()

In [ ]:
# Calculating player stability based on plus-minus
def opponent_stability_pm(df_player_opp_pm):
    return (
        df_player_opp_pm.groupby(["PLAYER_ID","PLAYER_NAME"], as_index=False)
          .agg(
              num_opponents_faced=("OPP_TEAM_ID","nunique"),
              pm_전체_평균=("pm_mean","mean"),
              pm_표준편차=("pm_mean","std"),
              pm_최소값=("pm_mean","min"),
              pm_최대값=("pm_mean","max"),
          )
    )
Champ_stability_pm = opponent_stability_pm(Champ_player_opp_pm_over2)
Other_stability_pm = opponent_stability_pm(Other_player_opp_pm_over2)

In [ ]:
Champ_stability_pm.head()

In [ ]:
Champ_pm_plot = Champ_stability_pm[["PLAYER_NAME", "pm_전체_평균", "pm_표준편차"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Champ_pm_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("우승 시즌 선수 코트 내 영향력 지표")
plt.tight_layout()
plt.show()


- 상대별 평균 PLUS_MINUS(pm_mean_overall)과 상대별 평균 PLUS_MINUS(pm_std)를 비교해봤을 때 상대적으로 pm_std가 큰 것을 볼 수 있음.
- 이는 선수의 활약이 상대팀이 달라짐에 따라 안정적이지 못한다고 해석해 볼 수 있음.

In [ ]:
# Plot to see the pm_min, pm_max comparison
Champ_pm_plot= Champ_stability_pm[["PLAYER_NAME", "pm_최소값", "pm_최대값"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Champ_pm_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("우승 시즌 상대별 최소, 최대 코트 영향력")
plt.tight_layout()
plt.show()

- 상대별 평균 PLUS_MINUS의 최소점과 최대점이 차이가 생각보다 크게 나는 것처럼 보인다.
- 이는 선수의 활약이 상대에 의존이 큰 매치업이 존재한다는 것을 볼 수 있다.

In [ ]:
Other_stability_pm.head()

In [ ]:
# Plot to see the pm_mean_overall comparison
Other_pm_plot= Other_stability_pm[["PLAYER_NAME", "pm_전체_평균", "pm_표준편차"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Other_pm_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("이전 시즌 선수 코트 내 영향력 지표")
plt.tight_layout()
plt.show()

- 상대별 평균 PLUS_MINUS(pm_mean_overall)과 상대별 평균 PLUS_MINUS(pm_std)를 비교해봤을 때 상대적으로 pm_std가 큰 것을 볼 수 있음.
- 이는 선수의 활약이 상대팀이 달라짐에 따라 안정적이지 못한다고 해석해 볼 수 있음.

In [ ]:
# Plot to see the pm_min, pm_max comparison
Other_pm_plot= Other_stability_pm[["PLAYER_NAME", "pm_최소값", "pm_최대값"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10, 6))
Other_pm_plot.T.plot(kind="bar", rot=0)
plt.ylabel("값")
plt.title("이전 시즌 상대별 최소, 최대 코트 영향력")
plt.tight_layout()
plt.show()

- 상대별 평균 PLUS_MINUS의 최소점과 최대점이 차이가 생각보다 크게 나는 것처럼 보인다.
- 이는 선수의 활약이 상대에 의존이 큰 매치업이 존재한다는 것을 볼 수 있다.

In [ ]:
# Compare Champ vs Other season player stability
compare_pm = (
    Champ_stability_pm.merge(
        Other_stability_pm,
        on=["PLAYER_ID","PLAYER_NAME"],
        how="inner",
        suffixes=("_champ","_other")
    )
)

compare_pm["delta_pm_표준편차"] = compare_pm["pm_표준편차_champ"] - compare_pm["pm_표준편차_other"]
compare_pm

In [ ]:
compare_pm_plot = compare_pm[["PLAYER_NAME","delta_pm_표준편차"]].set_index("PLAYER_NAME")
plt.figure(figsize=(10,6))
compare_pm_plot.T.plot(kind="bar", rot=0)
plt.axhline(0, linewidth=1)
plt.ylabel("표준편차 차이")
plt.title("우승 시즌과 이전 시즌 간 선수 코트 내 영향력 안정성 비교")
plt.tight_layout()
plt.show()

- 2015년 시즌과 이전 시즌을 비교했을 때 대부분 선수가 상대별 PLUS_MINUS의 분산이 증가했다고 알 수 있다.
- 이는 선수의 활약이 이전시즌과 비교해, 우승시즌에는 상대팀이 달라지면 안정적이지 못하다는 것을 볼 수 있다.

### 결론 인사이트
우승 시즌의 변화는 활약의 안정성 증가로 볼 수 없음을 확인할 수 있다.
오히려, 우승 시즌에는 성과가 상대팀에 따라 다르게 나타나는 경향으로 관찰되었다.

### 단장 관점에서 인사이트 연관
위의 분석을 통해 단장이 선수를 뽑을 때, 어떤 상대가 등장하든, 항상 잘하는 선수만 찾기보다는
특정 상대팀을 만났을 때 특히 강점을 보이는 선수들을 고려해보도록 조언을 할 수 있다.